In [1]:
import pandas as pd
import duckdb as db

In [2]:
con = db.connect("../../ingestao_de_dados/database/db_distribuicao_profissoes.db")

In [3]:
con.execute("SHOW TABLES").fetchdf()

,name
0,etl_estados
1,origens_advogados
2,origens_contadores
3,origens_engenheiros
4,origens_psicologos
5,tbl_advogados_brasil_2024
6,tbl_contadores_brasil_2024
7,tbl_engenheiros_brasil_2024
8,tbl_estados_brasil
9,tbl_psicologos_brasil_2024


In [7]:
tabelas = {
    "advogados": "origens_advogados",
    "contadores": "origens_contadores",
    "engenheiros": "origens_engenheiros",
    "psicologos": "origens_psicologos",
    "estados": "etl_estados"
}

tabela = {nome: con.execute(f"SELECT * FROM {tabela}").fetchdf() for nome, tabela in tabelas.items()}

In [ ]:
tabela['advogados']

In [ ]:
tabela['contadores']

In [ ]:
tabela['engenheiros']

In [ ]:
tabela['psicologos']

In [ ]:
tabela['estados']

In [21]:
con.execute("DESCRIBE origens_advogados").fetchdf()
con.execute("DESCRIBE origens_contadores").fetchdf()
con.execute("DESCRIBE origens_engenheiros").fetchdf()
con.execute("DESCRIBE origens_psicologos").fetchdf()

,column_name,column_type,null,key,default,extra
0,Estado,VARCHAR,YES,None,None,None
1,Feminino,BIGINT,YES,None,None,None
2,Masculino,BIGINT,YES,None,None,None


In [43]:
query = """
    SELECT
        'advogado' AS profissao,
        Estado,
        SUM(Feminino) AS feminino,
        SUM(Masculino) AS masculino
    FROM origens_advogados
    GROUP BY Estado

    UNION ALL

    SELECT
       'contador' AS profissao,
        Estado,
        SUM(Feminino),
        SUM(Masculino)
    FROM origens_contadores
    GROUP BY Estado

    UNION ALL

    SELECT
        'engenheiro' AS profissao,
        Estado,
        SUM(Feminino),
        SUM(Masculino)
    FROM origens_engenheiros
    GROUP BY Estado

    UNION ALL

    SELECT
        'psicologo' AS profissao,
        Estado,
        SUM(Feminino),
        SUM(Masculino)
    FROM origens_psicologos
    GROUP BY Estado;

"""

In [44]:
df_profissoes = con.execute(query).fetchdf()
df_profissoes

,profissao,Estado,feminino,masculino
0,advogado,DF,26508.0,25358.0
1,advogado,MA,12026.0,12020.0
2,advogado,MG,74205.0,69942.0
3,advogado,SP,197236.0,181116.0
4,advogado,CE,19541.0,20292.0
...,...,...,...,...
103,psicologo,Sergipe,3372.0,723.0
104,psicologo,Distrito Federal,13246.0,2407.0
105,psicologo,Goiás,13638.0,1966.0
106,psicologo,Mato Grosso do Sul,5204.0,789.0


In [49]:
df_profissoes[df_profissoes['profissao'] == 'advogado'].describe()

,feminino,masculino
count,27.000000,27.000000
mean,27508.518519,25522.851852
std,39997.618600,36525.059606
min,1473.000000,1398.000000
25%,7741.000000,7695.500000
50%,13516.000000,12059.000000
75%,27792.500000,25293.500000
max,197236.000000,181116.000000


In [50]:
df_profissoes[df_profissoes['profissao'] == 'contador'].describe()

,feminino,masculino
count,27.000000,27.000000
mean,6777.518519,7631.000000
std,9851.721413,11128.348709
min,512.000000,501.000000
25%,1976.000000,2207.000000
50%,4037.000000,4174.000000
75%,6521.000000,7952.000000
max,49302.000000,56528.000000


In [51]:
df_profissoes[df_profissoes['profissao'] == 'engenheiro'].describe()

,feminino,masculino
count,27.000000,27.000000
mean,8695.222222,34465.666667
std,11525.265271,56973.482517
min,391.000000,1119.000000
25%,2029.000000,7572.000000
50%,4602.000000,17778.000000
75%,10007.500000,33231.500000
max,53124.000000,281812.000000


In [52]:
df_profissoes[df_profissoes['profissao'] == 'psicologo'].describe()

,feminino,masculino
count,27.000000,27.000000
mean,16698.592593,2884.222222
std,26513.606301,4540.061041
min,971.000000,167.000000
25%,4584.000000,756.000000
50%,6371.000000,1206.000000
75%,17526.500000,2834.500000
max,132077.000000,22613.000000
